# Laboratorio 5 

Javier Valle 20159

In [68]:
from Archivos.clasificador import clasificacion
import pandas as pd
import json
import random
from pandas import json_normalize
import re

### Parte 1

Leyendo y guardando el tráfico que es DNS nada más

In [63]:
with open('./Archivos/large_eve.json', 'r') as archivo_json, \
        open('./Archivos/dns_trafico.json', 'w') as archivo_dns:
    
    for linea in archivo_json:
        datos = json.loads(linea)
        if datos['event_type'] == 'dns':
            json.dump(datos, archivo_dns)
            archivo_dns.write('\n')

archivo_json.close()
archivo_dns.close()

Enseñando la cantidad de registros

In [60]:
with open('./Archivos/dns_trafico.json', 'r') as archivo_dns:
    contador_registros = 0

    for linea in archivo_dns:
        contador_registros += 1

print("Cantidad de registros guardados:", contador_registros)

Cantidad de registros guardados: 15749


Enseñando la información de 2 registros cualesquiera

In [61]:
lineas = []

with open('./Archivos/dns_trafico.json', 'r') as archivo_dns:
    for linea in archivo_dns:
        lineas.append(linea)

# Escogiendo dos índices aleatorios dentro del rango de líneas en el archivo
indice1 = random.randint(0, len(lineas)-1)
indice2 = random.randint(0, len(lineas)-1)

# Obteniendo los datos de los registros correspondientes a los índices escogidos
registro1 = json.loads(lineas[indice1])
registro2 = json.loads(lineas[indice2])


print("Registro 1:")
print(json.dumps(registro1, indent=4)) 
print("\nRegistro 2:")
print(json.dumps(registro2, indent=4))


Registro 1:
{
    "timestamp": "2017-07-22T19:09:55.252660-0500",
    "flow_id": 1793277885143642,
    "pcap_cnt": 2624023,
    "event_type": "dns",
    "vlan": 140,
    "src_ip": "192.168.204.60",
    "src_port": 52268,
    "dest_ip": "128.244.177.34",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 9212,
        "rrname": "254.25.168.192.in-addr.arpa",
        "rrtype": "PTR",
        "tx_id": 2
    }
}

Registro 2:
{
    "timestamp": "2017-07-22T18:29:17.658269-0500",
    "flow_id": 254444630248086,
    "pcap_cnt": 505169,
    "event_type": "dns",
    "vlan": 150,
    "src_ip": "192.168.207.4",
    "src_port": 53,
    "dest_ip": "192.168.205.188",
    "dest_port": 56636,
    "proto": "UDP",
    "dns": {
        "type": "answer",
        "id": 39632,
        "rcode": "NXDOMAIN",
        "rrname": "mirror.clarkson.edu"
    }
}


Normalizando la data

In [41]:
with open('./Archivos/dns_trafico.json', 'r') as archivo_dns:
    datos = [json.loads(linea) for linea in archivo_dns]

# Normalizando los datos
df = pd.json_normalize(datos)

print("Shape del DataFrame:", df.shape)

Shape del DataFrame: (15749, 18)


Buscando dominios DGA

In [64]:
# Filtrando los registros DNS tipo A
registros_tipo_A = df[df['dns.rrtype'] == 'A']

print("Shape del DataFrame filtrado:", len(registros_tipo_A))

Shape del DataFrame filtrado: 2849


Filtrando los dominios únicos

In [66]:
# Conjunto para almacenar dominios únicos
unique_domains = set()

with open('./Archivos/dns_trafico.json', 'r') as archivo_dns:
    for linea in archivo_dns:
        datos = json.loads(linea)
        # Extrayendo el dominio del campo 'rrname' y agregarlo al conjunto
        domain = datos['dns']['rrname']
        unique_domains.add(domain)

num_unique_domains = len(unique_domains)

print("Cantidad de registros únicos:", num_unique_domains)

Cantidad de registros únicos: 334


Escribiendo una función que obtengo el TLD para un dominio

In [73]:
def obtener_TLD(dominio):
    # Expresión regular para encontrar el TLD
    patron = r'\.(.*?)\.(?:com|org|net|gov|edu|int|mil|arpa|biz|info|name|pro|aero|coop|museum|jobs|travel|cat|tel|onion|git|home|local|example|test|invalid|localhost|localdomain)$'
    match = re.search(patron, dominio)
    if match:
        return match.group(1)
    else:
        return dominio

# Ejemplo de uso
dominio1 = "api.wunderground.com"
dominio2 = "safebrowsing.clients.google.com.home"

print("TLD de", dominio1, ":", obtener_TLD(dominio1))
print("TLD de", dominio2, ":", obtener_TLD(dominio2))

TLD de api.wunderground.com : wunderground
TLD de safebrowsing.clients.google.com.home : clients.google.com


Prompt utilizado: "hola, me podrías escribir una función que me obtenga el TLD para un dominio"

Creando una columna que se llame domain_tld y eliminando las demás columnas

In [87]:
if 'domain_tld' not in df.columns:
    df['domain_tld'] = df['domain'].apply(obtener_TLD)
    
    df = df[['domain_tld']]

print(df)

                                    domain_tld
0      2001:0dbb:0c18:0011:0260:6eff:fe30:0863
1      2001:0dbb:0c18:0011:0260:6eff:fe30:0863
2                              192.168.205.170
3                                192.168.207.4
4                                192.168.207.4
...                                        ...
15744                            192.168.207.4
15745                            192.168.207.4
15746                           192.168.203.67
15747                            192.168.207.4
15748                            192.168.207.4

[15749 rows x 1 columns]
